In [1]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from lxml import etree
import time

# get player name
def get_player_name():
    player_list = []
    player = sp.find_all(class_ = 'full-3fV3c9pF')
    for i in range(0, len(player), 2):
        player_list.append(player[i].text + ' ' + player[i+1].text)
    return player_list

# get player's position
def get_player_position():
    position_list = []
    position = sp.find_all(class_ = 'position-28TbwVOg')
    for i in position:
        position_list.append(i.text)
    return position_list

def get_column_name():
    click_standard_button()
    sp = soup(driver.page_source, 'lxml')
    
    data_column_name = ['PLAYER', 'POSITION', 'TEAM']
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
    temp = sp.find_all(class_ = 'bui-text cellheader selected-1vxxHvFg bui-text')
    for i in range(0, len(temp), 2):
        data_column_name.append(temp[i].text)

    click_expanded_button()
    sp = soup(driver.page_source, 'lxml')
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
        
    return data_column_name

# click standard button
def click_standard_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    #time.sleep(0.5)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    time.sleep(2)
    sp = soup(driver.page_source, 'lxml')

# click expanded button
def click_expanded_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    #time.sleep(0.5)    
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    time.sleep(2)
    sp = soup(driver.page_source, 'lxml')

# click next page button
def click_next_page_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[3]/div[2]/div/div/div[2]/button').click()
    #time.sleep(0.5)
    #driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[3]/div[2]/div/div/div[2]/button').click()
    time.sleep(2)
    sp = soup(driver.page_source, 'lxml')

# get standard data
def get_data():
    data = []
    
    click_standard_button()
    sp = soup(driver.page_source, 'lxml')
    standard_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        standard_data.append(i.text)
    
    click_expanded_button()
    sp = soup(driver.page_source, 'lxml')
    expanded_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        expanded_data.append(i.text)

    for i in range(len(standard_data)//17):
        data.extend(standard_data[i*17:i*17+17])
        data.extend(expanded_data[i*16+1:i*16+16])

    return data

In [2]:
urls = pd.read_csv('mlb_stats_urls.csv', encoding='utf-8')

for year in range(2014, 2015):
    for link in urls['link']:
        df_list = []
        for page in range(1, 4):
            url = link + str(year) + '?page=' + str(page) + '&playerPool=ALL'
            driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')
            html = driver.get(url)
            sp = soup(driver.page_source, 'lxml')
            column_name = get_column_name()
            try:
                #while True:
                sp = soup(driver.page_source, 'lxml')
                print('page start')
                time.sleep(1)
                name = get_player_name()
                time.sleep(1)
                position = get_player_position()
                time.sleep(1)
                current_data = get_data()
                time.sleep(1)
                for i in range(len(name)):
                    row = []
                    row.append(name[i])
                    row.append(position[i])
                    row.extend(current_data[i*32:i*32+32])
                    df_list.append(row)
                time.sleep(1)
                #click_next_page_button()
            except:
                print('This is the last page :)')
        df = pd.DataFrame(df_list, columns=column_name)
        df.to_csv(f'mlb_stats/{year}/{link[26:-1]}_{year}.csv', index=False, encoding='utf-8')
        print(df)

C:\Users\user\AppData\Local\Temp\ipykernel_8732\2533676316.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
page start
              PLAYER POSITION TEAM    G   AB    R    H  2B 3B  HR  ... GO/AO  \
0         Mike Trout       CF  LAA  157  602  115  173  39  9  36  ...  0.58   
1      Albert Pujols       1B  LAA  159  633   89  172  37  1  28  ...  1.15   
2       Kole Calhoun       RF  LAA  127  493   90  134  31  3  17  ...  1.06   
3     Chris Iannetta        C  LAA  108  306   41   77  22  0   7  ...  0.79   
4     Gordon Beckham       3B  LAA   26   56   10   15   3  0   2  ...  1.00   
5      Josh Hamilton       LF  LAA   89  338   43   89  21  0  10  ...  0.91   
6     Howie Kendrick       2B  LAA  157  617   85  181  33  5   7  ...  2.09   
7          C.J. Cron       DH  LAA   79  242   28   62  12  1  11  ...  0.70   
8       David Freese       3B  LAA  134  462   53  120  25  1  10  ...  1.30   
9        Erick Aybar       SS  LAA  156  589   77  164  30  4   7  ...  1.25   
10    Collin Cowgill       RF  LAA  106  260   37   65  10  1   5  ...  1.05   
11     

C:\Users\user\AppData\Local\Temp\ipykernel_8732\2533676316.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
page start
               PLAYER POSITION TEAM    G   AB   R    H  2B 3B  HR  ... GO/AO  \
0       Samuel Deduno        P  HOU    5    1   0    1   1  0   0  ...  -.--   
1         Jose Altuve       2B  HOU  158  660  85  225  47  3   7  ...  1.05   
2     George Springer       RF  HOU   78  295  45   68   8  1  20  ...  1.02   
3          Max Stassi        C  HOU    7   20   2    7   2  0   0  ...  0.75   
4        Chris Carter       DH  HOU  145  507  68  115  21  1  37  ...  0.47   
5       Dexter Fowler       CF  HOU  116  434  61  120  21  4   8  ...  0.98   
6   Enrique Hernandez       CF  HOU   24   81  10   23   4  2   1  ...  0.74   
7     Marwin Gonzalez       SS  HOU  103  285  33   79  15  1   6  ...  1.49   
8      Gregorio Petit       SS  HOU   37   97  14   27   8  0   2  ...  1.05   
9     Carlos Corporan        C  HOU   55  170  22   40   6  0   6  ...  0.88   
10    Robbie Grossman       LF  HOU  103  360  42   84  14  2   6  ...  1.05   
11     

C:\Users\user\AppData\Local\Temp\ipykernel_8732\2533676316.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


...

page start
page start
page start
               PLAYER POSITION TEAM    G   AB    R    H  2B 3B  HR  ... GO/AO  \
0       Jorge Polanco       SS  MIN    5    6    2    2   1  1   0  ...  1.00   
1       Danny Santana       CF  MIN  101  405   70  129  27  7   7  ...  1.18   
2       Kennys Vargas       DH  MIN   53  215   26   59  10  1   9  ...  1.26   
3        Brian Dozier       2B  MIN  156  598  112  145  33  1  23  ...  0.68   
4       Oswaldo Arcia       RF  MIN  103  372   46   86  16  3  20  ...  0.88   
5      Trevor Plouffe       3B  MIN  136  520   69  134  40  2  14  ...  0.78   
6     Josh Willingham       LF  MIN   68  224   34   47   5  1  12  ...  0.49   
7           Joe Mauer       1B  MIN  120  455   60  126  27  2   4  ...  1.50   
8        Doug Bernier        X  MIN    7    7    2    2   0  0   0  ...  0.00   
9         Kurt Suzuki        C  MIN  131  452   37  130  34  0   3  ...  0.95   
10           Sam Fuld       CF  MIN   53  164   15   45  10  0   1  ...  1.3

C:\Users\user\AppData\Local\Temp\ipykernel_8732\2533676316.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
page start
              PLAYER POSITION TEAM    G   AB   R    H  2B  3B  HR  ... GO/AO  \
0         Jose Abreu       1B  CWS  145  556  80  176  35   2  36  ...  1.46   
1          Adam Dunn       DH  CWS  106  363  43   80  17   0  20  ...  0.87   
2         Adam Eaton       CF  CWS  123  486  76  146  26  10   1  ...  1.90   
3    Conor Gillaspie       3B  CWS  130  464  50  131  31   5   7  ...  0.74   
4      Moises Sierra       RF  CWS   83  127  20   35   8   2   2  ...  0.97   
5       Josh Phegley        C  CWS   11   37   4    8   2   0   3  ...  0.73   
6     Avisail Garcia       RF  CWS   46  172  19   42   8   0   7  ...  1.67   
7     Alexei Ramirez       SS  CWS  158  622  82  170  35   2  15  ...  1.02   
8      Tyler Flowers        C  CWS  127  407  42   98  16   1  15  ...  1.53   
9      Dayan Viciedo       RF  CWS  145  523  65  121  22   3  21  ...  1.08   
10     Marcus Semien       3B  CWS   64  231  30   54  10   2   6  ...  0.79   
11     

C:\Users\user\AppData\Local\Temp\ipykernel_8732\2533676316.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
page start
                PLAYER POSITION TEAM    G   AB   R    H  2B 3B  HR  ... GO/AO  \
0          Jose Pirela       2B  NYY    7   24   6    8   1  2   0  ...  1.40   
1         Martin Prado       2B  NYY   37  133  18   42   9  0   7  ...  0.70   
2          Chris Young       LF  NYY   23   71   9   20   8  0   3  ...  0.40   
3   Francisco Cervelli        C  NYY   49  146  18   44  11  1   2  ...  1.26   
4        Chase Headley       3B  NYY   58  191  28   50   8  0   6  ...  1.24   
5       Scott Sizemore       3B  NYY    6   16   3    5   2  0   0  ...  2.00   
6        Brett Gardner       LF  NYY  148  555  87  142  25  8  17  ...  0.87   
7      Jacoby Ellsbury       CF  NYY  149  575  71  156  27  5  16  ...  0.89   
8    Yangervis Solarte       3B  NYY   75  252  26   64  14  0   6  ...  1.00   
9        Mark Teixeira       1B  NYY  123  440  56   95  14  0  22  ...  1.00   
10      Carlos Beltran       DH  NYY  109  403  46   94  23  0  15  ...  1.0

In [ ]:
driver.quit()